In [12]:
# ==========================================================
# 🧩 LangChain Memory & Agent Demo (v0.2+ Compatible)
# ==========================================================
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain, SimpleSequentialChain, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.memory import ChatMessageHistory, ConversationSummaryBufferMemory, ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.vectorstores import Chroma, FAISS

# ----------------------------------------------------------
# Initialize LLM and Embeddings
# ----------------------------------------------------------
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
embeddings = OpenAIEmbeddings()


# ----------------------------------------------------------
# 🧠 Q1: Chatbot with ConversationSummaryBufferMemory
# ----------------------------------------------------------
summary_memory = ConversationSummaryBufferMemory(llm=llm)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

chain = prompt | llm

chat_history = ChatMessageHistory()
chat = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="history",
)

print(chat.invoke({"input": "I live in Chennai and work as ML Engineer."}, config={"configurable": {"session_id": "user1"}}).content)
print()
print(chat.invoke({"input": "What do you remember about me?"}, config={"configurable": {"session_id": "user1"}}).content)
print("\n\n")


# ----------------------------------------------------------
# 🪟 Q2: Chatbot with Limited Memory (Window of 3 messages)
# ----------------------------------------------------------
window_history = ChatMessageHistory()

def limited_history(session_id):
    msgs = window_history.messages[-6:]
    window_history.clear()
    for m in msgs:
        window_history.add_message(m)
    return window_history

chat_window = RunnableWithMessageHistory(
    chain,
    limited_history,
    input_messages_key="input",
    history_messages_key="history",
)

messages = ["I like football.", "I enjoy cricket.", "I work as data analyst.", "I play chess.", "I love Python."]
for msg in messages:
    print(chat_window.invoke({"input": msg}, config={"configurable": {"session_id": "user2"}}).content)
    print()

print(chat_window.invoke({"input": "What did I say first?"}, config={"configurable": {"session_id": "user2"}}).content)
print("\n\n")


# ----------------------------------------------------------
# 🧾 Q3: Simulated Knowledge Graph Memory
# ----------------------------------------------------------
facts = {}

def remember_facts(user_input):
    if "likes" in user_input:
        name, likes = user_input.split("likes")
        facts[name.strip()] = likes.strip()
        return f"Got it! Remembered that {name.strip()} likes {likes.strip()}."
    elif "What does" in user_input:
        name = user_input.split(" ")[2]
        return f"{name} likes {facts.get(name, 'Unknown')}."
    else:
        return "Tell me something new!"

print(remember_facts("Pavan likes cricket and Python."))
print()
print(remember_facts("What does Pavan like?"))
print("\n\n")


# ----------------------------------------------------------
# 👥 Q4: Entity Memory (Manual Entity Recall)
# ----------------------------------------------------------
entities = {"Pavan": "Chennai", "Ravi": "Mumbai"}
print(f"Where does Ravi live? Ravi lives in {entities['Ravi']}.")
print("\n\n")


# ----------------------------------------------------------
# ✍️ Q5: LLMChain with PromptTemplate
# ----------------------------------------------------------
template = "Write a 2-line poem about {topic}"
prompt = PromptTemplate(input_variables=["topic"], template=template)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({"topic": "AI in Healthcare"}).strip())
print("\n\n")


# ----------------------------------------------------------
# 🔗 Q6: SimpleSequentialChain Example
# ----------------------------------------------------------
chain1 = LLMChain(llm=llm, prompt=PromptTemplate(template="Generate a slogan for {product}", input_variables=["product"]))
chain2 = LLMChain(llm=llm, prompt=PromptTemplate(template="Expand the slogan: {slogan} into full ad copy", input_variables=["slogan"]))
seq_chain = SimpleSequentialChain(chains=[chain1, chain2])
print(seq_chain.run("Sunbucks Coffee").strip())
print("\n\n")


# ----------------------------------------------------------
# 🌍 Q7: Few-shot Translation PromptTemplate
# ----------------------------------------------------------
few_shot_template = """
Translate the following English words to French.
Hello -> Bonjour
Goodbye -> Au revoir
Now translate {word} ->
"""
few_shot_prompt = PromptTemplate(input_variables=["word"], template=few_shot_template)
translation_chain = LLMChain(llm=llm, prompt=few_shot_prompt)
print(translation_chain.run({"word": "Good night"}).strip())
print("\n\n")


# ----------------------------------------------------------
# 🧮 Q8: Agent with Calculator Tool
# ----------------------------------------------------------
def calculator_tool(query: str) -> str:
    try:
        return str(eval(query))
    except:
        return "Error in calculation"

tools = [Tool(name="Calculator", func=calculator_tool, description="Solves math expressions")]
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)
print(agent.invoke("Calculate 75*4 + 20")["output"])
print("\n\n")


# ----------------------------------------------------------
# 🌦️ Q9: Agent with Calculator + Weather Tool
# ----------------------------------------------------------
def weather_tool(query: str) -> str:
    return "Sunny in Chennai"

tools_weather = [
    Tool(name="Calculator", func=calculator_tool, description="Math"),
    Tool(name="Weather", func=weather_tool, description="Weather info")
]
agent_weather = initialize_agent(tools=tools_weather, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)
print(agent_weather.invoke("What is the weather in Chennai and calculate 10*5")["output"])
print("\n\n")


# ----------------------------------------------------------
# 🧠 Q10: Stateful Agent with Memory (Fixed)
# ----------------------------------------------------------
memory_agent = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_mem = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory_agent,
    verbose=False
)
print(agent_mem.invoke({"input": "My favorite color is Blue", "chat_history": []})["output"])
print()
print(agent_mem.invoke({"input": "What is my favorite color?", "chat_history": []})["output"])
print("\n\n")


# ----------------------------------------------------------
# 📚 Q11: Agent with RAG Tool (Chroma)
# ----------------------------------------------------------
docs = ["Pavan is a Machine Learning Engineer in Chennai."]
vectorstore = Chroma.from_texts(docs, embedding=embeddings, persist_directory="chroma_rag")
retriever = vectorstore.as_retriever()

def rag_search(q):
    docs = retriever.get_relevant_documents(q)
    return "\n\n".join([d.page_content for d in docs])

rag_tool = Tool(name="RAG Search", func=rag_search, description="Search documents using vectorstore")
agent_rag = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)
print(agent_rag.invoke("Who is Pavan and where is he from?")["output"])
print("\n\n")


# ----------------------------------------------------------
# 🧮 Q12: Agent with Memory + RAG + Calculator
# ----------------------------------------------------------
agent_rag_mem = initialize_agent(
    tools=[rag_tool, Tool(name="Calculator", func=calculator_tool, description="Math")],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=False
)
print(agent_rag_mem.invoke({"input": "My name is Pavan", "chat_history": []})["output"])
print()
print(agent_rag_mem.invoke({"input": "Where do I live?", "chat_history": []})["output"])
print()
print(agent_rag_mem.invoke({"input": "Calculate 12*12", "chat_history": []})["output"])
print("\n\n")


# ----------------------------------------------------------
# 🕉️ Q13: Retrieval QA Chain (Chroma)
# ----------------------------------------------------------
temple_docs = ["Jagannath Temple is in Puri, Odisha.", "Meenakshi Temple is in Madurai, Tamil Nadu."]
temple_store = Chroma.from_texts(temple_docs, embedding=embeddings, persist_directory="temple_rag")
temple_retriever = temple_store.as_retriever()
temple_qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=temple_retriever)
print(temple_qa.invoke({"question": "Where is the Jagannath temple?", "chat_history": []})["answer"])


# ----------------------------------------------------------
# 🧭 Q14: FAISS Retrieval
# ----------------------------------------------------------
faiss_store = FAISS.from_texts(["Sunbucks is a coffee chain."], embedding=embeddings)
faiss_retriever = faiss_store.as_retriever()
faiss_qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=faiss_retriever)
print(faiss_qa.invoke({"question": "Which coffee chain is mentioned?", "chat_history": []})["answer"])
print("\n\n")


# ----------------------------------------------------------
# 💾 Q15: Multi-document RAG Retrieval
# ----------------------------------------------------------
doc_texts = ["Python is a programming language.", "SQL is used for databases.", "Python can be used for ML."]
doc_store = Chroma.from_texts(doc_texts, embedding=embeddings, persist_directory="ml_docs")
doc_retriever = doc_store.as_retriever()
doc_qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=doc_retriever)
print(doc_qa.invoke({"question": "Which language is used for ML?", "chat_history": []})["answer"])


# ----------------------------------------------------------
# 🔤 Q16: Agent with Translator + Calculator (Fixed chat_history)
# ----------------------------------------------------------
translator_tool = Tool(
    name="Translator", 
    func=lambda x: "Bonjour" if "Hello" in x else "Unknown", 
    description="Translate English to French"
)
agent_dynamic = initialize_agent(
    tools=[translator_tool, Tool(name="Calculator", func=calculator_tool, description="Math")],
    llm=llm, 
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
    verbose=False
)
print(agent_dynamic.invoke({"input": "Translate 'Hello' to French", "chat_history": []})["output"])
print(agent_dynamic.invoke({"input": "Calculate 50*7", "chat_history": []})["output"])


That's great! Chennai has a growing tech scene, and as an ML Engineer, you have a lot of opportunities to work on exciting projects. Are you currently working on any specific machine learning projects, or are you looking for resources, networking opportunities, or anything else related to your field?

I don’t have the ability to remember personal information about users or past interactions. Each conversation is treated independently, and I don’t retain any details once the session ends. How can I assist you today?



That's great! Football is a popular sport enjoyed by millions around the world. Do you have a favorite team or player? Or is there a particular league or competition you follow?

That's great to hear! Cricket is a fascinating sport with a rich history and a passionate following around the world. Do you have a favorite team or player? Or is there a particular format of the game you enjoy, like Test matches, One Day Internationals, or T20?

That's great! As a data analyst, 

In [13]:
# ==========================================================
# 🧩 Q17: Multi-tool Agent with Dynamic Prompt & Memory
# ==========================================================
dynamic_tools = [
    Tool(name="Calculator", func=calculator_tool, description="Math expressions"),
    Tool(name="Translator", func=lambda x: "Bonjour" if "Hello" in x else "Unknown", description="English -> French"),
    rag_tool
]

dynamic_agent = initialize_agent(
    tools=dynamic_tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=False
)

print(dynamic_agent.invoke({"input": "Translate 'Hello' to French and calculate 12*12", "chat_history": []})["output"])
print(dynamic_agent.invoke({"input": "Who is Pavan?", "chat_history": []})["output"])
print("\n\n")


# ==========================================================
# 🧠 Q18: Conversational Agent with Summary Memory
# ==========================================================
summary_memory_agent = initialize_agent(
    tools=[rag_tool, Tool(name="Calculator", func=calculator_tool, description="Math")],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history"),
    verbose=False
)

print(summary_memory_agent.invoke({"input": "I work in Chennai as ML Engineer", "chat_history": []})["output"])
print(summary_memory_agent.invoke({"input": "Where do I work?", "chat_history": []})["output"])
print("\n\n")


# ==========================================================
# 📚 Q19: Multi-document RAG Retrieval with Follow-up
# ==========================================================
multi_docs = [
    "Python supports machine learning and AI.",
    "SQL is used for database management and analytics.",
    "Pandas and NumPy are Python libraries for data analysis."
]

multi_doc_store = Chroma.from_texts(multi_docs, embedding=embeddings, persist_directory="multi_doc_rag")
multi_retriever = multi_doc_store.as_retriever()
multi_doc_qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=multi_retriever)

print(multi_doc_qa.invoke({"question": "Which language is used for ML?", "chat_history": []})["answer"])
print(multi_doc_qa.invoke({"question": "Which Python libraries are used for data analysis?", "chat_history": []})["answer"])
print("\n\n")


# ==========================================================
# 🔗 Q20: Agent + Multi-document RAG + Calculator + Translator
# ==========================================================
combined_tools = [
    Tool(name="Calculator", func=calculator_tool, description="Math"),
    Tool(name="Translator", func=lambda x: "Bonjour" if "Hello" in x else "Unknown", description="English->French"),
    Tool(name="Multi-doc RAG", func=lambda q: "\n".join([d.page_content for d in multi_retriever.get_relevant_documents(q)]), description="Search multi-documents")
]

combined_agent = initialize_agent(
    tools=combined_tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=False
)

print(combined_agent.invoke({"input": "Translate 'Hello' to French, calculate 20*5 and find info on Python ML libraries", "chat_history": []})["output"])


The translation of "Hello" to French is "Bonjour," and the result of 12 multiplied by 12 is 144.
Pavan is a common name in India and can refer to various individuals. Without more context, it's difficult to determine which specific Pavan you are referring to. It could be a person in popular culture, a public figure, or someone in your personal life. Could you provide more details?
```



That's great to hear! Working as a Machine Learning Engineer in Chennai must be exciting, given the city's growing tech scene. If you have any questions about machine learning, projects you're working on, or anything else, feel free to ask!
```
You mentioned that you work in Chennai as a Machine Learning Engineer. If you have any specific questions or topics you'd like to discuss related to your work, feel free to share!
```



Python is commonly used for machine learning (ML).
Pandas and NumPy are Python libraries used for data analysis.



Here are the results of your requests:
1. "Hello" in French i